# Classification Experiment - Lexicality (Language vs. Non-language)
By: Shateesh Bhugwansing



### Experiment 1 - Logistic Regression

__Data__: EpochedEEG/20131216_1441_epo.fif (Epoched data from Emmanuil's batch script)

__Problem__: Classify language vs. Non-language epochs 

__Procedure__:
1. filter the epoched data for epochs 1-24 
2. label events by either language or non-language, using the dictionary that Emmanuil created (preprocessing/StimCodes
3. Run Logistic Regression classifier on the labeled data. 


In [1]:
%run ../preprocessing/StimCodes.ipynb
import mne
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams.update({'font.size': 11})

{'AALL': [(5, 6, 4), (17, 18, 16)],
 'AALN': [(11, 6, 10), (23, 18, 22)],
 'AANL': [(5, 12, 4), (17, 24, 16)],
 'AANN': [(11, 12, 10), (23, 24, 22)],
 'AVLL': [(2, 6, 1), (14, 18, 13)],
 'AVNN': [(8, 12, 7), (20, 24, 19)],
 'VALL': [(5, 3, 4), (17, 15, 16)],
 'VANN': [(11, 9, 10), (23, 21, 22)],
 'VVLL': [(2, 3, 1), (14, 15, 13)],
 'VVLN': [(8, 3, 7), (20, 15, 19)],
 'VVNL': [(2, 9, 1), (14, 21, 13)],
 'VVNN': [(8, 9, 7), (20, 21, 19)]}